In [2]:
import pandas as pd

# 1. Read training data

In [3]:
df = pd.read_csv("data/9_clean_training_data/clean_training_data.csv")

2. Define a model

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 250, 500],
    'min_samples_leaf': [1, 2, 4],
    'criterion': ['mse', 'friedman_mse']
}
variable = 'gwl_cm'
temporal_expl = [
    'VV', 'VH', 'VVVH_ratio', 
    'precipitation', 'prec_3', 'prec_7', 'prec_30', 
    'sm_1',  'sm_3', 'sm_7', 'sm_30'
]
row = {}

for station_id in df['id'].unique():

    train_data = df[df['id'] != station_id]
    test_data = df[df['id'] == station_id]
    X_train, X_test = train_data[explain_vars], test_data[explain_vars]
    y_train, y_test = train_data[variable], test_data[variable]

    regr = RandomForestRegressor(random_state=42, oob_score=True, n_jobs=-1)
    grid_search = GridSearchCV(estimator=regr, param_grid=param_grid, cv=3)
    grid_search.fit(X_train, y_train)
    best_regr = grid_search.best_estimator_
    
    y_pred = best_regr.predict(X_test)

    r, _ = pearsonr(test_data[variable].values, y_pred)
    correlations = [r]
    correlations.extend([test_data[variable].corr(test_data[expl]) for expl in temporal_expl])
    row[station_id] = correlations

stats_df = pd.DataFrame.from_dict(row, orient='index')


In [ ]:
variable = "gwl_cm"
row = {}
temporal_expl = []
training_df = df[df.region_id == 2].copy()
temporal_expl = [
    'VV', 'VH', 'VVVH_ratio', 
    'precipitation', 'prec_3', 'prec_7', 'prec_30', 
    'sm_1',  'sm_3', 'sm_7', 'sm_30'
]


for i, station_id in enumerate(training_df.id.unique()):
    
    
    # add first element gwl-smm correlation to station_id
    explans = []
    
    # create a df for training without the station_id
    gdf_train = training_df[training_df.id != station_id]
    
    # create a df for test of the station_id
    gdf_test = training_df[training_df.id == station_id]
        
    X_train, X_test = gdf_train[explain_vars], gdf_test[explain_vars]
    y_train, y_test = gdf_train[variable], gdf_test[variable]
    
    regr = RandomForestRegressor(n_estimators=250, min_samples_leaf=1, random_state=42, oob_score=True, criterion='friedman_mse', n_jobs=-1)
    regr.fit(X_train, y_train)
    y_pred_test = regr.predict(X_test)
    
    r, p = pearsonr(gdf_test[variable].values, y_pred_test)
    explans.append(r)
    
    #r2 = r2_score(y_test, y_pred_test)
    #explans.append(r2)
    
    # add correlation of explanatories
    for expl in temporal_expl:
        explans.append(gdf_test[variable].corr(gdf_test[expl]))
        
    row[station_id] = explans
    
stats_df = pd.DataFrame.from_dict(row, orient='index')

In [ ]:
stats_df.columns = ["friedman_mse"] + temporal_expl

In [ ]:
from matplotlib import pyplot as plt


stats_df.loc['mean'] = stats_df.mean()
sns.heatmap(stats_df, annot=True)

figures_path = Path("data/8_figures/correlations/")
figures_path.mkdir(parents=True, exist_ok=True)

plt.savefig(figures_path/'correlation_by_group_1.png')